In [1]:
# VGG Model based on https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
# Data augmentation based on https://chatbotslife.com/using-augmentation-to-mimic-human-driving-496b569760a9

import sys
import numpy as np
import cv2
import sklearn.metrics as sklm

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.datasets import mnist

from keras import backend as K
img_dim_ordering = 'tf'
K.set_image_dim_ordering(img_dim_ordering)

from PIL import Image
import glob
import os
from matplotlib.pyplot import imshow
import numpy as np
import copy
%matplotlib inline

Using TensorFlow backend.


In [2]:
current = os.path.dirname(os.path.abspath('__file_'))

In [3]:
# Flipping red images
def flip(_image):
    return cv2.flip(_image,1)

In [4]:
# Rotate images
def trans_image(_image,trans_range):    
    rows, cols, chan = _image.shape      
    tr_x = trans_range#*np.random.uniform()-trans_range/2    
    Trans_M = np.float32([[1,0,tr_x],[0,1,0]])
    image_tr = cv2.warpAffine(_image,Trans_M,(cols,rows))    
    return image_tr

In [5]:
# Brightness
def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image1 = np.array(image1, dtype = np.float64)
    random_bright = .5+np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1[:,:,2][image1[:,:,2]>255]  = 255
    image1 = np.array(image1, dtype = np.uint8)
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

In [6]:
# Shadow
def add_random_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    #random_bright = .25+.7*np.random.uniform()
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    
    return image

In [7]:
def read_in(_current,directory, function):
    path = os.path.join(_current, directory)
    print(path)
    image_list_ = []
    for filename in glob.glob(path+'/*.png'): 
        im=Image.open(filename).convert('RGB')    
        if function == 'flip':
            im = flip(np.array(im))
        elif function == 'trans':
            im = trans_image(np.array(im),20)
        elif function == 'bright':        
            im = augment_brightness_camera_images(np.array(im))
        elif function == 'shadow':
            add_random_shadow(np.array(im))
        else:
            im = im            
        opencvImage = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)    
        new_width  = 224
        new_height = 224
        im = cv2.resize(opencvImage, (224, 224),interpolation = cv2.INTER_CUBIC).astype(np.float32)
        im[:,:,0] -= 103.939
        im[:,:,1] -= 116.779
        im[:,:,2] -= 123.68
            #im = im.transpose((2,0,1))    
            #im = cv2.resize(opencvImage,(new_width, new_height), interpolation = cv2.INTER_CUBIC)    
        image_list_.append(im)
    print(len(image_list_))
    return image_list_ 

#image_list_red = read_in(current, '_0_','shadow')
#print(len(image_list_red))
#imshow(image_list_red[9])

In [8]:
image_list_red_norm = read_in(current, '_0_','')
image_list_red_flip = read_in(current, '_0_','flip')
image_list_red_trans = read_in(current, '_0_','trans')
image_list_red_bright = read_in(current, '_0_','bright')
image_list_red_shadow = read_in(current, '_0_','shadow')

val_red = np.concatenate([image_list_red_norm, image_list_red_flip,image_list_red_trans,image_list_red_bright,image_list_red_shadow])
print(len(val_red))
lab_red = np.zeros(len(val_red))
print(len(lab_red))

D:\Programming\Python\tensorflow-vgg\_0_
1598
D:\Programming\Python\tensorflow-vgg\_0_
1598
D:\Programming\Python\tensorflow-vgg\_0_
1598
D:\Programming\Python\tensorflow-vgg\_0_
1598
D:\Programming\Python\tensorflow-vgg\_0_
1598
7990
7990


In [ ]:
image_list_yellow_norm = read_in(current, '_1_','')
image_list_yellow_flip = read_in(current, '_1_','flip')
image_list_yellow_trans = read_in(current, '_1_','trans')
image_list_yellow_bright = read_in(current, '_1_','bright')
image_list_yellow_shadow = read_in(current, '_1_','shadow')

val_yellow = np.concatenate([image_list_yellow_norm, image_list_yellow_flip,image_list_yellow_trans,image_list_yellow_bright,image_list_yellow_shadow])
print(len(val_yellow))
lab_yellow = np.ones(len(val_yellow))
print(len(lab_yellow))

D:\Programming\Python\tensorflow-vgg\_1_
285
D:\Programming\Python\tensorflow-vgg\_1_
285
D:\Programming\Python\tensorflow-vgg\_1_
285
D:\Programming\Python\tensorflow-vgg\_1_
285
D:\Programming\Python\tensorflow-vgg\_1_
285
1425
1425


In [ ]:
image_list_green_norm = read_in(current, '_2_','')
image_list_green_flip = read_in(current, '_2_','flip')
image_list_green_trans = read_in(current, '_2_','trans')
image_list_green_bright = read_in(current, '_2_','bright')
image_list_green_shadow = read_in(current, '_2_','shadow')

val_green = np.concatenate([image_list_green_norm, image_list_green_flip,image_list_green_trans,image_list_green_bright,image_list_green_shadow])
print(len(val_green))
lab_green = np.empty(len(val_green))
lab_green.fill(2)

D:\Programming\Python\tensorflow-vgg\_2_
1223
D:\Programming\Python\tensorflow-vgg\_2_
1223
D:\Programming\Python\tensorflow-vgg\_2_
1223
D:\Programming\Python\tensorflow-vgg\_2_
1223
D:\Programming\Python\tensorflow-vgg\_2_
1223


In [ ]:
image_list_none_norm = read_in(current, '_4_','')
image_list_none_flip = read_in(current, '_4_','flip')
image_list_none_trans = read_in(current, '_4_','trans')
image_list_none_bright = read_in(current, '_4_','bright')
image_list_none_shadow = read_in(current, '_4_','shadow')

val_none = np.concatenate([image_list_none_norm, image_list_none_flip,image_list_none_trans,image_list_none_bright,image_list_none_shadow])
print(len(val_none))
lab_none = np.empty(len(val_none))
lab_none.fill(4)

In [ ]:
val_tot = np.concatenate([val_red,val_yellow,val_green,val_none])                         
lab_tot = np.concatenate([lab_red , lab_yellow, lab_green, lab_none])

In [ ]:
# the model
def pretrained_model(img_shape, num_classes, layer_type):
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
    #model_vgg16_conv.summary()
    for layer in model_vgg16_conv.layers:        
        layer.trainable = False
    
    #Create your own input format
    keras_input = Input(shape=img_shape, name = 'image_input')
    
    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(keras_input)
    
    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation=layer_type, name='fc1')(x)
    x = Dense(400, activation=layer_type, name='fc2')(x)
    x = Dense(num_classes, activation='softmax', name='predictions')(x)
    
    #Create your own model 
    pretrained_model = Model(input=keras_input, output=x)
    pretrained_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return pretrained_model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(val_tot, lab_tot, test_size=0.33, random_state=42, shuffle= True)

In [ ]:
# training the model
model = pretrained_model(x_train.shape[1:], len(set(y_train)), 'relu')

In [ ]:
hist = model.fit(x_train, y_train, batch_size = 1000, nb_epoch= 20, validation_data=(x_test, y_test), verbose=1)